In [2]:
# import libraries
from docxtpl import DocxTemplate
from docx import Document
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.enum.section import WD_SECTION, WD_ORIENTATION
from docx.shared import RGBColor, Mm, Pt, Inches
from docx.oxml import OxmlElement
from docx.oxml.ns import qn
from docx.enum.table import WD_ROW_HEIGHT_RULE
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
from urllib.parse import urljoin
import json
from datetime import date, datetime as dt

c:\Users\Gabriel O\Desktop\ACL\ACL\Knowledge Base\Python_Reports\report_temp\Lib\site-packages\docxcompose\properties.py:18: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [4]:
####################################################################################
# fetch data
####################################################################################

BEARER_TOKEN = 'd358c3e1709f951fe611415e7a0b1956b783127d52d541adc48f649761408f15'
org_id = '11534' #'52734'
id = '492704'
server_url = 'https://apis-us.diligentoneplatform.com'
BASE_URL = f'https://apis-us.diligentoneplatform.com/v1/orgs/{org_id}/projects/{id}'
headers = {'Authorization' : f'Bearer {BEARER_TOKEN}', 'Content-Type': 'application/vnd.api+json'}
params = {'filter[status]':'active', 'filter[start_date][lte]': dt.today().isoformat(), 'include':'project, project.fieldwork'}

def fetch_data(url, params=None, max_retries=5, timeout=10, backoff_factor=1):
    """
        fetch data from endpoint with maxixum retries, timeout and session management

        Args:
            url(str): targeted endpoint
            max_retries(int): no of retry attempt upon non-response from the server
            timeout(int): prevent prolonged server connection
            backoff_factor = wait multiplier between request
    """
    retries = Retry(total= max_retries, 
                    backoff_factor=backoff_factor, 
                    status_forcelist=[429, 500, 502, 503, 504], #retries on these http errors
                    allowed_methods=['GET'])
    session = requests.session()
    adapter = HTTPAdapter(max_retries=retries)
    session.mount('http://',adapter)
    session.mount('https://', adapter)
    all_data = []
    while url:
        try:
            resp = session.get(url, headers=headers, params=params, timeout=10)
            resp.raise_for_status()
        except Exception as e:
            print(f'failure to fetch data: {e}')
        project_data = resp.json().get('data', [])
        all_data = all_data + project_data if isinstance(project_data, list) else project_data # check and accumulate multiple project or return a single targerted project
        next_page = resp.json()['links']['next'] if isinstance(project_data, list) else None
        url = urljoin(server_url,next_page) if next_page else None
        params = None
    session.close()
    return all_data

In [ ]:
# fetch valid project data ***************************************************************
projects = fetch_data('BASE_URL')
projects

{'id': '492704',
 'type': 'projects',
 'attributes': {'name': 'SURTECH TEST -Gabe testing',
  'state': 'active',
  'status': 'active',
  'created_at': '2025-08-26T09:26:02Z',
  'updated_at': '2025-08-26T12:37:16Z',
  'description': '',
  'background': '<p>As part of the Surtech independent review process, Internal Audit and Risk Function conducted a Test Audit. This is included in the Approved 2023/2024 Annual Internal Audit Plan.</p><p>Internal Audit and Risk Function’s work predominantly uses the risk-based approach. That is, the team obtains an understanding of the work that the auditable section does, areas that expose the Company to risks are identified and then audit responses are developed to test if the controls put in place are operating effectively in mitigating the identified risks.\xa0</p>',
  'budget': None,
  'position': 0,
  'certification': False,
  'control_performance': False,
  'risk_assurance': True,
  'management_response': '',
  'max_sample_size': 25,
  'number_of

In [5]:
url = 'https://cts.projects.diligentoneplatform.com/audits/492704/objectives/1896423/controls/14421654'
resp = requests.get(url, headers=headers)
resp.status_code

419